# Projet guidé - Jeu de cartes Coloretto

Ce projet guidé va vous faire utiliser la plupart des concepts vus jusqu’à présent.

Nous allons implémenter ensemble un jeu de cartes appelé Coloretto.

Les règles du jeu :

- [en français](https://ludos.brussels/ludo-cocof/opac_css/doc_num.php?explnum_id=105)
- [en anglais](https://www.riograndegames.com/wp-content/uploads/2013/02/Coloretto-Rules.pdf)

## Créer une classe représentant un joueur

In [93]:
from collections import Counter


class Player:
    def __init__(self, name, cpu=True):
        self.name = name
        self.cpu = cpu
        self.cards = []

    def take(self, cards):
        """method to take a card stack and add them to the player's cards"""
        self.cards.extend(cards)

    def score(self, cards):
        """With a list of cards, compute the score."""
        # build a score table based on quantities gathered
        quantities = range(1, 7)
        scores = [1, 3, 6, 10, 15, 21]
        score_table = dict(zip(quantities, scores))

        # initialise score with +2 cards
        score = cards.count("+2") * 2

        # count the jokers
        jokers = cards.count("Joker")

        # create a new list with only the normal cards to evaluate
        cards_to_score = [card for card in cards if card != "+2" and card != "Joker"]

        # Count each color
        counter = Counter(cards_to_score)

        # trim quantities above 6, they bring nothing extra
        for color, qty in counter.most_common():
            if qty > 6:
                counter[color] = 6

        # add jokers where they help the most
        for j in range(jokers):
            for color, qty in counter.most_common():
                if qty < 6:
                    counter[color] += 1
                    break
        # set positives and negatives
        if len(counter) >= 3:
            positives = counter.most_common()[:3]
        else:
            positives = counter.most_common()

        if len(counter) > 3:
            negatives = counter.most_common()[3:]
        else:
            negatives = []

        ##print(f"{counter=}")
        # compute positives and negatives contribution
        #print(f"{score=}")
        #print(f"{positives=}")
        for color, qty in positives:
            score += score_table.get(qty)
        #print(f"Après compte des positifs, {score=}")
        #print(f"{negatives=}")
        for color, qty in negatives:
            score -= score_table.get(qty)
        #print(f"Après déccompte des négatifs, {score=}")
        return score
    
    def score_current_cards(self):
        return self.score(self.cards)

    def __repr__(self):
        return f"Player('{self.name}')"

In [78]:
p1 = Player(name="Phil")

In [79]:
p1.cards = (
    ["Red"] * 3
    + ["Orange"] * 12
    + ["Blue"] * 2
    + ["Yellow"] * 4
    + ["Purple"] * 1
    + ["Joker"] * 2
)

In [80]:
p1.cards

['Red',
 'Red',
 'Red',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Orange',
 'Blue',
 'Blue',
 'Yellow',
 'Yellow',
 'Yellow',
 'Yellow',
 'Purple',
 'Joker',
 'Joker']

In [81]:
p1.score_current_cards()

counter=Counter({'Orange': 6, 'Yellow': 6, 'Red': 3, 'Blue': 2, 'Purple': 1})
score=0
positives=[('Orange', 6), ('Yellow', 6), ('Red', 3)]
Après compte des positifs, score=48
negatives=[('Blue', 2), ('Purple', 1)]
Après déccompte des négatifs, score=44


44

## Demander au joueur le nombre de joueurs (entre 3 et 5)

Nous allons demander à l’utilisateur le nombre de joueurs souhaités.

Si le nombre de joueurs est de 3, nous retirons une des couleurs.

Enfin nous allons constituer le paquet de cartes.

In [82]:
num_players = 0
while num_players not in range(3, 6):
    answer = input("Choisissez un nombre de joueurs entre 3 et 5…\n")
    try:
        num_players = int(answer)
        if num_players < 3:
            print("Nombre de joueurs trop petit.")
        if num_players > 5:
            print("Nombre de joueurs trop grand.")
    except:
        print("Votre réponse n’est pas valide.")
        num_players = 0

Choisissez un nombre de joueurs entre 3 et 5…
 Phil


Votre réponse n’est pas valide.


Choisissez un nombre de joueurs entre 3 et 5…
 5


In [60]:
import random

In [83]:
colors = ["Red", "Green", "Blue", "Orange", "Yellow", "Gray", "Purple"]

if num_players == 3:
    colors.remove(random.choice(colors))

In [84]:
colors

['Red', 'Green', 'Blue', 'Orange', 'Yellow', 'Gray', 'Purple']

In [85]:
deck = colors * 9 + ["+2"] * 10 + ["Joker"] * 3
random.shuffle(deck)
deck[:10]

['Red', 'Purple', 'Joker', 'Yellow', '+2', 'Blue', 'Green', '+2', '+2', 'Red']

Créons les différents joueurs

In [86]:
player_name = None
while not player_name:
    player_name = input('What is your name?\n')
    if not player_name:
        print('invalid name, retry')

What is your name?
 Phil


In [87]:
bot_names = ['Mr Handy', 'Cylon', 'Astro', 'Bishop', 'Goldorak', 'Defect', 'Silica', 'Prospero']

In [95]:
p1 = Player(name=player_name, cpu=False)

In [96]:
cpus = [Player(name) for name in random.sample(bot_names, k=num_players - 1)]

In [97]:
cpus

[Player('Goldorak'), Player('Astro'), Player('Silica'), Player('Defect')]

In [100]:
players = [p1] + cpus
random.shuffle(players)
players

[Player('Goldorak'),
 Player('Phil'),
 Player('Silica'),
 Player('Astro'),
 Player('Defect')]

Il est temps de créer la boucle principale de notre jeu.

D’après les règles du jeu, on insère une carte une carte spéciale à la 16ème place à partir de la fin du paquet de cartes. Cet artifice n'est pas nécessaire dans notre programme, on continuera simplement à démarrer des rounds **tant que** notre paquet de carte est plus long que 15.

In [101]:
round_counter = 0
while len(deck) > 15:
    round_counter += 1
    print(f"ROUND {round_counter}")
    deck = deck[15:]

ROUND 1
ROUND 2
ROUND 3
ROUND 4
ROUND 5
